In [2]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.decomposition import TruncatedSVD

view_log_train = pd.read_csv('../data/view_log.csv')
article_info = pd.read_csv('../data/article_info.csv')
submission = pd.read_csv('../submission/sample_submission.csv')

In [3]:
view_log_train.head(100)

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
95,USER_0005,ARTICLE_0564,MG,BR
96,USER_0005,ARTICLE_0931,MG,BR
97,USER_0005,ARTICLE_2223,MG,BR
98,USER_0005,ARTICLE_1577,MG,BR


In [4]:
# 사용자-아이템 행렬 생성
view_log_train.drop(['userRegion', 'userCountry'], axis=1, inplace=True)
df = view_log_train
user_item_matrix = df.pivot_table(index='userID', columns='articleID', aggfunc='size', fill_value=0)

In [7]:
df

,userID,articleID
0,USER_0000,ARTICLE_0661
1,USER_0000,ARTICLE_2316
2,USER_0000,ARTICLE_1345
3,USER_0000,ARTICLE_1089
4,USER_0000,ARTICLE_1484
...,...,...
42712,USER_1420,ARTICLE_0682
42713,USER_1420,ARTICLE_2179
42714,USER_1420,ARTICLE_1848
42715,USER_1420,ARTICLE_0030


In [91]:
from sklearn.decomposition import TruncatedSVD

# 사용자-아이템 행렬 생성
user_item_matrix = df.pivot_table(index='userID', columns='articleID', aggfunc='size', fill_value=0)

# SVD 모델 생성
svd = TruncatedSVD(n_components=50, random_state=42)
user_item_matrix_svd = svd.fit_transform(user_item_matrix)

In [92]:
def get_svd_recommendations(user_id, n_recommendations=5):
    user_index = user_item_matrix.index.tolist().index(user_id)
    user_vector = user_item_matrix_svd[user_index]
    # 모든 기사에 대해 점수 계산
    scores = np.dot(user_item_matrix_svd, user_vector)
    # 상위 n개의 기사 추천
    recommended_indices = np.argsort(scores)[::-1][:n_recommendations]
    return user_item_matrix.columns[recommended_indices].tolist()



In [93]:
# 예시: USER_0001에 대한 추천
recommended_articles_svd = get_svd_recommendations('USER_0001', n_recommendations=5)
print(f"Recommended articles for USER_0001 (SVD): {recommended_articles_svd}")

Recommended articles for USER_0001 (SVD): ['ARTICLE_0553', 'ARTICLE_1385', 'ARTICLE_1101', 'ARTICLE_0843', 'ARTICLE_1131']


In [ ]:
|